In [1]:
import torch

Load matrix $Q$

In [2]:
mat_q = torch.load('mat_q_resid_excluded_missing.pt')
mat_q = torch.abs(mat_q)
mat_q.shape

torch.Size([32064, 472])

Construct a correlation coefficient matrix
$$
R(i, j)=\frac{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))(z(s_j,t_k)-\tilde{z}(s_j))}{\sqrt{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))^2}\sqrt{\sum_{k=1}^d(z(s_j,t_k)-\tilde{z}(s_j))^2}},
$$
where $$\tilde{z}(s_i)=\frac{1}{d}\sum_{k=1}^dz(s_i,t_k)$$

In [3]:
d, p = mat_q.shape
mat_q_normalized = mat_q - mat_q.mean(dim=0)
mat_r = torch.zeros(p, p)
for i in range(p):
    for j in range(i, p):
        i_col = mat_q_normalized[:, i]
        j_col = mat_q_normalized[:, j]
        i_norm = max(torch.norm(i_col), 1e-12)
        j_norm = max(torch.norm(j_col), 1e-12)
        if i_norm == 0 or j_norm == 0:
            if i_norm == j_norm:
                mat_r[i][j] = 1
            else:
                mat_r[i][j] = 0
        else:
            mat_r[i][j] = (i_col @ j_col) / i_norm / j_norm
        mat_r[j][i] = mat_r[i][j]
mat_r = mat_r / mat_r.max()

torch.save(mat_r, 'mat_r_excluded_missing.pt')
mat_r.shape

torch.Size([472, 472])

Load matrix R

In [4]:
mat_r = torch.load('mat_r_excluded_missing.pt')
mat_r.shape

torch.Size([472, 472])

In [5]:
ALPHA = 0.9999

n_timesteps, n_sections = mat_q.shape
n_grouped = 0
groups = []

mat_r_copy = mat_r - torch.diag(mat_r.diag())
while n_grouped < n_sections:
    new_group_idx = torch.nonzero(mat_r_copy > ALPHA)
    if len(new_group_idx) > 0:
        corr = mat_r[new_group_idx[:, 0], new_group_idx[:, 1]]
        new_group_idx = new_group_idx[:, 0].unique()

        n_grouped += len(new_group_idx)
        mat_r_copy[new_group_idx, :] = 0
        mat_r_copy[:, new_group_idx] = 0
        groups.append((new_group_idx, corr.min(), corr.max()))
        if mat_r_copy.max() == 0:
            break
        else:
            mat_r_copy /= mat_r_copy.max()
del mat_r_copy

Analyse grouping

In [6]:
print(f'Using alpha={ALPHA}, {n_sections} correlated sections were divided ' +
      f'into {len(groups)} groups:')
n_ungrouped = 0
for i, (group, corr_min, corr_max) in enumerate(groups, start=1):
      group_coeffs = mat_r[tuple(group.T), :]
      print(f'Group {i} - {len(group)} sections with correlation coefficients '
          f'{corr_min:.3f} to {corr_max:.3f}')
print(str(n_sections - n_grouped) + " section(s) don't correlate with anything and weren't grouped")


Using alpha=0.9999, 472 correlated sections were divided into 154 groups:
Group 1 - 2 sections with correlation coefficients 1.000 to 1.000
Group 2 - 2 sections with correlation coefficients 0.970 to 0.970
Group 3 - 2 sections with correlation coefficients 0.969 to 0.969
Group 4 - 2 sections with correlation coefficients 0.261 to 0.261
Group 5 - 2 sections with correlation coefficients 0.259 to 0.259
Group 6 - 2 sections with correlation coefficients 0.252 to 0.252
Group 7 - 2 sections with correlation coefficients 0.252 to 0.252
Group 8 - 2 sections with correlation coefficients 0.249 to 0.249
Group 9 - 2 sections with correlation coefficients 0.248 to 0.248
Group 10 - 2 sections with correlation coefficients 0.246 to 0.246
Group 11 - 2 sections with correlation coefficients 0.244 to 0.244
Group 12 - 2 sections with correlation coefficients 0.244 to 0.244
Group 13 - 2 sections with correlation coefficients 0.242 to 0.242
Group 14 - 2 sections with correlation coefficients 0.241 to 0.2

C:\Users\19ak9\AppData\Local\Temp\ipykernel_56520\3453684837.py:5: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3281.)
  group_coeffs = mat_r[tuple(group.T), :]


In [7]:
representatives = torch.stack([g[0] for g, _, _ in groups])
mat_c = mat_q[:, representatives]
assert mat_c.shape == (mat_q.shape[0], len(groups))
torch.save(mat_c, 'mat_c_excluded_missing.pt')
mat_c.shape

torch.Size([32064, 154])

In [8]:
mat_x = torch.linalg.pinv(mat_c) @ mat_q
torch.save(mat_x, 'mat_x_excluded_missing.pt')
mat_x.shape

torch.Size([154, 472])

In [9]:
print(torch.max((mat_c @ mat_x) - mat_q))

tensor(365.1183)
